In [29]:
import cv2
import os
import re
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import csv
import pandas as pd
import keras
from keras import models,Model
from keras.layers import Dense,GaussianNoise, Dropout,Input
%matplotlib inline
from skimage.feature import hog

In [30]:
dataSet = []
person = []
horiSet = []

count = 0
folder = "/home/jeet/CV Assignment/ML face images train Sketch"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
#         print(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200,200))
        fd, img = hog(img,visualise=True)
        img = np.ravel(img)
        if img is not None:
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person.append(numbers[0])
            horiSet.append(img.T)
            dataSet.append(img)
            count = count + 1
           
        
dataSet_test = []
person_test = []
horiSet_test = []

folder = "/home/jeet/CV Assignment/ML face images test Sketch translation"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        fd, img = hog(img, visualise=True)
        img = np.ravel(img)
        if img is not None:
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person_test.append(numbers[0])
            horiSet_test.append(img.T)
            dataSet_test.append(img)
            count = count + 1

/home/jeet/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [31]:
#Getting data in desired format and mapping the lables
dataSet = np.array(dataSet)
train = pd.DataFrame(dataSet)

person = np.array(person)
train['id'] = person.transpose()

unique_person = train['id']
unique_person = np.array(unique_person)
unique_person = np.unique(unique_person)
key = range(len(unique_person))

roll_dict = dict( zip(unique_person, key))

mapped_roll = np.vectorize(roll_dict.get)(person)
train['id'] = mapped_roll.transpose()

y = train['id'].values.astype('int64')
images = train.drop(['id'], axis=1, inplace=False)
x = (images.values).astype('uint8')

X_train = x
Y_train = y

dataSet_test = np.array(dataSet_test)
test = pd.DataFrame(dataSet_test)

person_test = np.array(person_test)
test['id'] = person_test.transpose()
print(test)

mapped_roll_test = np.vectorize(roll_dict.get)(person_test)
test['id'] = mapped_roll_test.transpose()

y_test = test['id'].values.astype('int64')
images = test.drop(['id'], axis=1, inplace=False)
x_test = (images.values).astype('uint8')

X_test = x_test
Y_test = y_test

      0    1    2    3    4    5    6    7    8    9    ...      39991  39992  \
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0    0.0   
11  0.0  0.0  0.0  0.0  0.0 

In [32]:
# img1 = cv2.imread('/home/jeet/ML face images test/201501067O_disgust.jpg')
# img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
# img1 = cv2.resize(img1, (200,200))
# fd, img = hog(img1,visualise=True)
# img = np.ravel(img)

min_arr = []
distance = []

for test_data in X_test:
    distance = []
#     print("\n In distance: ",distance)
    for train_data in X_train:
        dist = np.linalg.norm(test_data-train_data)
        distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
    min_arr.append(distance.index(np.min(distance)))
#     min_arr.append(np.min(distance))

In [33]:
print(len(min_arr))

68


In [34]:
print(Y_test)

[25  6 18 21 24 28 10  9 23 15 12  6 21  8 26 34  4  1 10  1 22 17 13 27
 33 29 17 16 27 11  7 11 24 30 25 12 19  2 16 35 29  3 28  3 23 22  0 32
  7  9 32 30 20  2  4 31 15 19 33 18 20 31 34  8 35 26 14 13]


In [35]:
Y_train

array([30, 34, 35, 20, 10,  3, 21, 21,  1, 15,  0, 15, 24, 16, 14,  2, 26,
       15,  5, 12, 21,  8, 35, 10, 23, 34, 32, 22,  6,  9, 31, 18,  9,  1,
       11, 33, 25,  8, 12,  9, 17,  4, 29, 15, 16, 13, 12,  0, 31, 26, 24,
       35,  8,  7,  0, 29,  4,  6, 35, 29, 34, 19, 24, 20, 28,  6, 28,  6,
       26, 33, 33, 31, 12, 26,  0, 15, 24, 13, 15, 11, 10, 28, 33, 23,  4,
       27, 19, 24,  5, 12, 32,  1, 32, 30,  2, 23,  7,  8, 14, 22, 11, 21,
        4, 16, 23, 31,  6, 25,  1,  1, 26,  4, 29,  5,  0, 27, 10, 13,  3,
       27, 19, 32, 30, 30, 14,  9, 13, 27, 17,  0, 25,  5, 34, 25,  7, 29,
        1,  2, 11, 32, 33, 34, 25, 35, 17,  4, 22, 10, 17, 33,  2, 22, 28,
       11, 22,  8, 25, 13,  7, 10, 20, 19, 20, 19,  3, 16, 31,  9,  8, 20,
       31,  6, 30, 21,  2, 30, 24, 16, 10, 23,  2, 16, 28, 26, 32,  3, 15,
        8,  7, 12, 18, 13, 29, 18, 20, 34, 17, 17, 18, 19, 23,  3, 18,  9,
       22, 11, 18, 35,  6, 28, 27, 14,  7, 27])

In [8]:
img1 = cv2.imread('/home/jeet/ML face images test/201501067O_disgust.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
img1 = cv2.resize(img1, (200,200))
fd, img = hog(img1,visualise=True)
img = np.ravel(img)

min_arr = []
distance = []

for train_data in X_train:
    dist = np.linalg.norm(img-train_data)
    distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
    min_arr.append(distance.index(np.min(distance)))
#     min_arr.append(np.min(distance))

/home/jeet/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [9]:
print(distance.index(np.min(distance)))

191


In [10]:
Y_train[389]

IndexError: index 389 is out of bounds for axis 0 with size 204

In [ ]:
Y_test

In [36]:
folder = "/home/jeet/CV Assignment/ML face images test Sketch translation"

count=0
for filename in os.listdir(folder):
        distance=[]
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        fd, img = hog(img, visualise=True)
        img = np.ravel(img)
        print("file : ",filename)
        
        for train_data in X_train:
            dist = np.linalg.norm(img-train_data)
            distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
            min_arr.append(distance.index(np.min(distance)))
        dis = distance.index(np.min(distance))
        print(Y_train[dis])
        
      
        for roll in roll_dict:
            if str(roll) in filename:
                print(roll)
#                 print(roll_dict[roll])
                
                if roll_dict[roll] == Y_train[dis]:
                    count = count+1
                    
print(count)

/home/jeet/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


file :  201501067_disgust.jpg
29
201501067
file :  201501004_disgust.JPG
29
201501004
file :  201501038_fear.jpg
18
201501038
file :  201501053_fear.jpg
2
201501053
file :  201501060_fear.jpg
24
201501060
file :  201501077_fear.jpg
24
201501077
file :  201501011_disgust.jpg
22
201501011
file :  201501009_fear.jpg
29
201501009
file :  201501055_fear.jpg
24
201501055
file :  201501031_disgust.jpg
24
201501031
file :  201501021_fear.jpg
24
201501021
file :  201501004_fear.JPG
6
201501004
file :  201501053_disgust.jpg
15
201501053
file :  201501008_disgust.jpg
24
201501008
file :  201501070_disgust.jpg
2
201501070
file :  201501104_fear.jpg
24
201501104
file :  1849004_fear.jpg
22
1849004
file :  1744002_disgust.jpg
16
1744002
file :  201501011_fear.jpg
24
201501011
file :  1744002_fear.jpg
1
1744002
file :  201501054_disgust.jpg
16
201501054
file :  201501034_disgust.jpg
2
201501034
file :  201501025_disgust.jpg
22
201501025
file :  201501071_fear.jpg
29
201501071
file :  201501097_disgus

In [37]:
roll_dict

{1744001: 0,
 1744002: 1,
 1744003: 2,
 1744004: 3,
 1849004: 4,
 20180119: 5,
 201501004: 6,
 201501007: 7,
 201501008: 8,
 201501009: 9,
 201501011: 10,
 201501012: 11,
 201501021: 12,
 201501025: 13,
 201501028: 14,
 201501031: 15,
 201501032: 16,
 201501034: 17,
 201501038: 18,
 201501039: 19,
 201501051: 20,
 201501053: 21,
 201501054: 22,
 201501055: 23,
 201501060: 24,
 201501067: 25,
 201501070: 26,
 201501071: 27,
 201501077: 28,
 201501086: 29,
 201501088: 30,
 201501091: 31,
 201501095: 32,
 201501097: 33,
 201501104: 34,
 201501109: 35}